READ DATASET

In [10]:
import pandas as pd

In [11]:
df = pd.read_excel("src/dataset.xlsx")

PREPROCESSING TEXT

In [12]:
import string
import re
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [13]:
def case_folding(sentence):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F700-\U0001F77F"  
        u"\U0001F780-\U0001F7FF"  
        u"\U0001F800-\U0001F8FF"  
        u"\U0001F900-\U0001F9FF"  
        u"\U0001FA00-\U0001FA6F"  
        u"\U0001FA70-\U0001FAFF"  
        u"\U0001F004-\U0001F0CF"  
        "]+", flags=re.UNICODE)

    sentence = emoji_pattern.sub(r'', sentence)
    sentence = sentence.translate(str.maketrans("","", string.punctuation)).lower()
    sentence = re.sub(r"\d+", "", sentence)
    sentence = sentence.replace("/", " ")
    return sentence

In [14]:
file = "abbreviation_file.txt"

def load_abbreviation_file(file):
    try:
        with open(file, "r") as file:
            abbreviations = json.load(file)
        return abbreviations
    except FileExistsError:
        print(f"File not found {file}")
        return {}

abbreviation_file = load_abbreviation_file(file)

def normalize_text(sentence):
    words = sentence.lower().split()
    words_normalized = []
    for word in words:
        for full_form, abbreviations in abbreviation_file.items():
            if word.lower() in abbreviations:
                words_normalized.append(full_form)
                break
        else:
            words_normalized.append(word)
    return " ".join(words_normalized)

In [15]:
def stopwords_removal(sentence):
    tokens = word_tokenize(sentence)
    liststopwords =  set(stopwords.words('indonesian'))
    custom_stopwords_file = "more_stopwords.txt"

    custom_stopwords = set()
    with open(custom_stopwords_file, "r") as file:
        for line in file:
            custom_stopwords.add(line.strip())

    combined_stopwords = liststopwords.union(custom_stopwords)

    with open(custom_stopwords_file, "w") as file:
        for word in combined_stopwords:
            file.write(word + "\n")

def remove_custom_stopwords(sentence, custom_stopwords_file):
    custom_stopwords = set()
    with open(custom_stopwords_file, 'r') as file:
        for line in file:
            custom_stopwords.add(line.strip())

    words = word_tokenize(sentence)

    filtered_words = [word for word in words if word.lower() not in custom_stopwords]

    cleaned_text = ' '.join(filtered_words)

    return cleaned_text

In [16]:
def stemming_text(sentence):
    factory = StemmerFactory()
    Stemmer = factory.create_stemmer()

    sentence = Stemmer.stem(sentence)
    return sentence

In [17]:
df['case_folding'] = df['review_text'].apply(case_folding)
df['normalized_text'] = df['case_folding'].apply(normalize_text)
df['stopword_removed'] = df['normalized_text'].apply(lambda x: remove_custom_stopwords(x, 'more_stopwords.txt'))
df['stemmed_text'] = df['stopword_removed'].apply(stemming_text)
pd.set_option('display.max_colwidth', None)
df.head(1)

,author_title,review_rating,review_text,case_folding,normalized_text,stopword_removed,stemmed_text
0,Felix zhang,5,"Urus KTP hilang cukup surat keterangan hilang dari kepolisian dan fotokopi KK\nAntrian sekitar 1 jam, proses cetak hanya sekitar 2 menit",urus ktp hilang cukup surat keterangan hilang dari kepolisian dan fotokopi kk\nantrian sekitar jam proses cetak hanya sekitar menit,urus kartu tanda penduduk hilang cukup surat keterangan hilang dari kepolisian dan fotokopi kartu keluarga antrian sekitar jam proses cetak hanya sekitar menit,urus kartu tanda penduduk hilang surat keterangan hilang kepolisian fotokopi kartu keluarga antrian jam proses cetak menit,urus kartu tanda duduk hilang surat terang hilang polisi fotokopi kartu keluarga antri jam proses cetak menit


In [19]:
import tensorflow
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import models